<a href="https://colab.research.google.com/github/whxodus0121/CSE3081-Algorithm-design-and-analysis/blob/main/%EC%BA%A1%EC%8A%A4%ED%86%A4%EB%94%94%EC%9E%90%EC%9D%B8_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# test1

In [ ]:
%%capture
!pip install datasets
!pip install accelerate>=0.20.3 transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install jamo
!pip install noisereduce

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
from datasets import load_dataset, load_metric, Audio

dataset = load_dataset("kresnik/zeroth_korean", "clean")
test_data = dataset['test']
train_data = dataset['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for kresnik/zeroth_korean contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kresnik/zeroth_korean
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`

In [ ]:
test_data

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 457
})

In [ ]:
train_data = train_data.remove_columns(["speaker_id", "chapter_id", "id"])
test_data = test_data.remove_columns(["speaker_id", "chapter_id", "id"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_data.remove_columns(["file", "audio"]), num_examples=10)

,text
0,민주주의자라면 그리고 좋은 지도자라면 계파나 개인이 아닌 정당이 집권해야 한다고 생각해야 한다
1,최근 민간에서 전자담배가 충전 도중 폭발하는 사고가 잇따라 당국이 주의를 당부하고 있는 가운데 병영 내 폭발 사고는 처음이다
2,대한변협은 당시 수사에 관여한 경찰들로부터 직무상범죄를 자인받았고 그 중 한 명에게 확인서를 작성받아 재판부에 증거로 제출했다
3,예를 들어 어떤 충실한 인물에 대해 읽는다면 그 사람의 특성과 행동을 그가 받은 축복과 연관 지어 보시기 바랍니다
4,모 대학에서 근무하는 그는 서울이 아니면 직장생활하기 어렵다며 미리 유산 준다고 생각하고 아파트를 해 달라고 목소리를 높였다
5,체중이 이상적인 정도보다 삼십 퍼센트 이상 더 나가는 사람은 고혈압이 생기기가 쉽습니다
6,실손보험에 가입했다면 대다수 정형외과에서 일 만 에서 이 만원의 적은 돈으로 치료를 받을 수 있다
7,이 회장에게 불거진 성매매 의혹 동영상 사건은 세계 굴지의 기업 총수와 관련된 사건인 만큼 검찰과 경찰이 투트랙으로 수사할지 아니면 검찰이 직접 맡아 진행할지 관심이 쏠렸다
8,연골이 퇴화되고 마모되면서 연골과 붙어 있는 뼈가 받는 압력이 증가한다고 그들은 생각합니다
9,혜원 신윤복이 그린 미인도 속 여인은 그 복식이나 치장을 통해 사대부가의 여인이 아닌 기생임을 유추할 수 있다


In [ ]:
import jamo
def decompose_hangul(batch):
  batch["text"] = jamo.j2hcj(jamo.h2j(batch["text"]))
  return batch

In [ ]:
train_data = train_data.map(decompose_hangul)
test_data = test_data.map(decompose_hangul)

In [ ]:
show_random_elements(train_data.remove_columns(["file","audio"]))

,text
0,ㄱㅗㅇㅂㅜㄹㅡㄹ ㅎㅏㄱㅓㄴㅏ ㄱㅕㅇㅎㅓㅁㅇㅡㄹ ㅎㅏㅁㅕㄴ ㅅㅣㄴㄱㅕㅇㅅㅔㅍㅗㅇㅢ ㅇㅕㄴㄱㅕㄹㅁㅏㅇㅇㅣ ㅈㅐㅈㅗㅈㅣㄱㄷㅚㄱㅓㄴㅏ ㅇㅕㄴㄱㅕㄹ ㄱㅏㅇㄷㅗㄱㅏ ㅂㅏㄲㅟㅁㅕㄴㅅㅓ ㄱㅣㅇㅓㄱㅇㅣㄴㅏ ㅍㅏㄴㄷㅏㄴ ㄴㅡㅇㄹㅕㄱㅇㅣ ㅅㅐㅇㄱㅣㄴㄷㅏ
1,ㅅㅗㄷㅡㄱ ㅅㅜㅈㅜㄴㅇㅣ ㄴㅗㅍㅇㅏㅈㅣㅁㅕㄴ ㅈㅓㄴㅊㅔ ㄱㅘㅇㅣㄹ ㅅㅗㅂㅣㄹㅑㅇㅇㅣ ㄴㅡㄹㅈㅣㅁㅏㄴ ㅅㅓㄴㅎㅗㅎㅏㄴㅡㄴ ㄱㅘㅇㅣㄹㄷㅗ ㄷㅏㅇㅑㅇㅎㅐㅈㅣㄴㄷㅏ
2,ㄱㅡㄴㅡㄴ ㅎㅏㄴ ㅇㅣㄹ ㅇㅚㄱㅛㄱㅘㄴㄱㅖㄴㅡㄴ ㄱㅜㄱㄱㅏㅇㅣㄹㅇㅣㄴㅣ ㅈㅓㅇㅂㅜㄱㅏ ㅎㅐㅇㅑㅎㅏㄴㅡㄴㄷㅔ ㄱㅜㅊㅓㅇㅈㅏㅇㅇㅔㄱㅔ ㅇㅛㄱㅜㅎㅏㄴㄷㅏㅁㅕㄴ ㅅㅏㅇㅇㅟ ㄷㅏㄴㅊㅔㄱㅏ ㅂㅣㄱㅓㅂㅎㅏㄴ ㄱㅓㅅㅇㅣㄹㅏㅁㅕ ㄱㅕㅇㅊㅏㄹㄱㅝㄴㄷㅗ ㅇㅚㄱㅛㄱㅝㄴㄷㅗ ㅇㅓㅄㄴㅡㄴ ㄱㅜㅊㅓㅇㅈㅏㅇㅇㅣ ㅁㅜㅅㅡㄴ ㅎㅣㅁㅇㅣ ㅇㅣㅆㄴㅡㄴㅑㅁㅕㄴㅅㅓ ㅁㅗㄱㅅㅗㄹㅣㄹㅡㄹ ㄴㅗㅍㅇㅕㅆㄷㅏ
3,ㄱㅡㄹㅓㄴㅏ ㅂㅏㄱㄱㅡㄴㅎㅖ ㄷㅐㅌㅗㅇㄹㅕㅇㅇㅣ ㅈㅣㄴㅏㄴㅎㅐ ㅅㅏㄴㅈㅣㄱㅘㄴㄱㅘㅇ ㅎㅘㄹㅅㅓㅇㅎㅘㄹㅡㄹ ㅇㅟㅎㅐ ㄱㅠㅈㅔ ㅇㅘㄴㅎㅘㄹㅡㄹ ㅈㅣㅅㅣㅎㅏㄴ ㅇㅣㄹㅐ ㅈㅣㄴㅊㅓㄱ ㅅㅗㄱㄷㅗㄱㅏ ㅃㅏㄹㄹㅏㅈㅕㅆㄷㅏ
4,ㅎㅏㅈㅣㅁㅏㄴ ㅇㅕㅈㅓㄴㅎㅣ ㅁㅏㄶㅇㅡㄴ ㅅㅏㄹㅏㅁㅇㅡㄴ ㅇㅣㅅㅣㅂ ㅇㅣㄹ ㅅㅔㄱㅣㅇㅢ ㅂㅗㄱㅈㅏㅂㅎㅏㄴ ㅅㅐㅇㅎㅘㄹㅇㅔㅅㅓ ㅂㅓㅅㅇㅓㄴㅏ ㅍㅕㄴㅇㅏㄴㅎㅏㄴ ㄱㅣㅊㅏㄹㅡㄹ ㅌㅏㄱㅗ ㅌㅗㄹㅗㄴㅌㅗㅇㅔㅅㅓ ㅂㅐㄴㅋㅜㅂㅓㄲㅏㅈㅣ ㅍㅕㄹㅊㅕㅈㅣㄴ ㅇㅏㄹㅡㅁㄷㅏㅇㅜㄴ ㄱㅕㅇㅊㅣㄹㅡㄹ ㄱㅏㅁㅅㅏㅇㅎㅏㄴㅡㄴ ㄱㅓㅅㅇㅡㄹ ㅈㅗㅎㅇㅏㅎㅏㅂㄴㅣㄷㅏ
5,ㅁㅜㄹㄹㅓㄴㅏㄴㅡㄴ ㄱㅓㅅ ㅃㅐㄱㅗㄴㅡㄴ ㅁㅝㄷㅡㄴ ㄷㅏ ㅂㅏㄲㅜㄹ ㅅㅜ ㅇㅣㅆㄴㅡㄴ ㄱㅓ ㅇㅏㄴㅣㄴㄱㅏ
6,ㅇㅣ ㅌㅡㄱㅂㅕㄹㄱㅏㅁㅊㅏㄹㄱㅘㄴㅇㅡㄴ ㄱㅜ ㅇㅝㄹ ㅅㅏㅁㅅㅣㅂ ㅇㅣㄹ ㅂㅓㅂㅅㅏㅇㅟㅇㅔ ㅈㅡㅇㅇㅣㄴㅇㅡㄹㅗ ㅊㅜㄹㅅㅓㄱㅎㅏㄹ ㅇㅖㅈㅓㅇㅇㅣㅇㅓㅆㄷㅏ
7,ㄱㅡ ㅅㅓㅇㄱㅜㅇㅔㅅㅓㄴㅡㄴ ㄱㅗㄷ ㅎㅏㄴㅡㄴㅣㅁㄲㅔㅅㅓ ㄸㅏㅇㅇㅡㄹ ㅍㅏㅁㅕㄹㅅㅣㅋㅣㄴㅡㄴ ㅈㅏㄷㅡㄹㅇㅡㄹ ㅍㅏㅁㅕㄹㅅㅣㅋㅣㅅㅣㄹ ㄱㅓㅅㅇㅣㄹㅏㄱㅗ ㅇㅏㄹㄹㅕ ㅈㅜㅂㄴㅣㄷㅏ
8,ㄱㅗㅁㅍㅏㅇㅇㅣㄴㅡㄴ ㅈㅣㄴㄱㅠㄴㄹㅠㅇㅔ ㅅㅗㄱㅎㅏㄴㅡㄴㄷㅔ ㅅㅣㅂ ㅁㅏㄴ ㅈㅗㅇㅇㅣ ㄴㅓㅁㄴㅡㄴ ㅈㅣㄴㄱㅠㄴㄹㅠㅇㅔㄴㅡㄴ ㅎㅢㄴㄱㅗㅁㅍㅏㅇㅇㅣ ㅂㅓㅅㅓㅅ ㄴㅗㄱㅂㅕㅇㄱㅠㄴ ㅎㅛㅁㅗ ㄷㅡㅇㅇㅣ ㅇㅣㅆㅅㅡㅂㄴㅣㄷㅏ
9,ㅎㅘㅇ ㄱㅝㄴㅎㅏㄴㄷㅐㅎㅐㅇㅇㅢ ㄷㅐㅅㅓㄴ ㅊㅜㄹㅁㅏㄹㅡㄹ ㅂㅏㄴㄷㅐㅎㅏㄴㅡㄴ ㅇㅣㅂㅈㅏㅇㅇㅣㄴ ㅇㅑㄷㅏㅇ ㅇㅕㄱㅅㅣ ㅎㅘㅇ ㄱㅝㄴㅎㅏㄴㄷㅐㅎㅐㅇㅇㅢ ㅎㅐㅇㅂㅗ ㄷㅐㅂㅜㅂㅜㄴㅇㅡㄹ ㄷㅐㄱㅝㄴㅎㅐㅇㅂㅗㄹㅗ ㅍㅜㄹㅇㅣㅎㅏㅁㅕ ㅇㅕㄴㅇㅣㄹ ㅁㅐㅇㄱㅗㅇㅇㅡㄹ ㅎㅏㄱㅗ ㅇㅣㅆㄴㅡㄴ ㅅㅏㅇㅎㅘㅇㅇㅣㄷㅏ


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_data.column_names)
vocab_test = test_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_data.column_names)

Map:   0%|          | 0/22263 [00:00<?, ? examples/s]

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ㅋ': 0,
 'ㅈ': 1,
 'ㅀ': 2,
 'ㅆ': 3,
 'ㅚ': 4,
 'ㄺ': 5,
 'ㅅ': 6,
 'ㅄ': 7,
 'ㅣ': 8,
 'ㄸ': 9,
 'ㄹ': 10,
 'ㅍ': 11,
 'ㅙ': 12,
 'ㅕ': 13,
 'ㅛ': 14,
 'ㅇ': 15,
 'ㅢ': 16,
 'ㄼ': 17,
 'ㅓ': 18,
 'ㄶ': 19,
 'ㅘ': 20,
 'ㄷ': 21,
 'ㅂ': 22,
 'ㄴ': 23,
 'ㅑ': 24,
 'ㄻ': 25,
 'ㅏ': 26,
 'ㄲ': 27,
 'ㅁ': 28,
 'ㅐ': 29,
 'ㄳ': 30,
 'ㅜ': 31,
 'ㅔ': 32,
 'ㅝ': 33,
 'ㅠ': 34,
 'ㄵ': 35,
 'ㅗ': 36,
 'ㅟ': 37,
 ' ': 38,
 'ㅖ': 39,
 'ㅃ': 40,
 'ㅒ': 41,
 'ㅞ': 42,
 'ㄱ': 43,
 'ㅎ': 44,
 'ㅡ': 45,
 'ㅊ': 46,
 'ㅌ': 47,
 'ㅉ': 48}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

51

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

ValueError: Mountpoint must not already contain files

In [ ]:
processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-test")

[]

In [ ]:
train_data[0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/160/160_003_0016.flac'

In [ ]:
train_data[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/160/160_003_0016.flac',
 'array': array([-3.05175781e-05, -3.05175781e-05, -3.05175781e-05, ...,
         3.05175781e-05,  0.00000000e+00,  0.00000000e+00]),
 'sampling_rate': 16000}

In [ ]:
train_data = train_data.cast_column("audio", Audio(sampling_rate=16_000))
test_data = test_data.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
train_data[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3c93119fdbcba519e1529c416a7339ed19b67c18686fc5bea5eda3309e01e58e/train_data_01/003/160/160_003_0016.flac',
 'array': array([-3.05175781e-05, -3.05175781e-05, -3.05175781e-05, ...,
         3.05175781e-05,  0.00000000e+00,  0.00000000e+00]),
 'sampling_rate': 16000}

In [ ]:
import noisereduce as nr

def remove_noise(batch):
    audio = batch["audio"]
    noisy_signal = audio["array"]
    reduced_noise = nr.reduce_noise(y=noisy_signal, sr=audio["sampling_rate"])
    batch["audio"]["array"] = reduced_noise
    return batch

train_data = train_data.map(remove_noise, num_proc=4)
test_data = test_data.map(remove_noise, num_proc=4)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_data)-1)

ipd.Audio(data=train_data[rand_int]["audio"]["array"], autoplay=True, rate=16000)

print("Target text:", train_data[rand_int]["text"])
print("Input array shape:", train_data[rand_int]["audio"]["array"].shape)
print("Sampling rate:", train_data[rand_int]["audio"]["sampling_rate"])

Target text: ㄱㅡㄹㅓㅁㅕㄴ ㅎㅗㅇㅅㅡㅇㅎㅢㄷㅗ ㅇㅣㅅㅓㄱㄱㅣ ㅊㅜㅈㅗㅇㅅㅔㄹㅕㄱㅇㅣㅈㅣ ㅇㅓㄱㅇㅜㄹㅎㅏㅁㅇㅡㄹ ㅇㅣㅎㅐㅎㅏㄴㅡㄴ ㄱㅓㅅㄱㅘ ㅊㅜㅈㅗㅇㅎㅏㄴㅡㄴ ㄱㅓㄴ ㄷㅏㄹㅡㄴ ㄱㅓㅅ ㄱㅏㅌㅇㅡㄴㄷㅔ
Input array shape: (113408,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names, num_proc=4)
test_data = test_data.map(prepare_dataset, remove_columns=test_data.column_names, num_proc=4)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/22263 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/457 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
max_input_length_in_sec = 5.0
train_data = train_data.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/22263 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-33-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    #"facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1914: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  #output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-test",
  group_by_length=True,
  per_device_train_batch_size=64,
  gradient_accumulation_steps=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=1000,
  eval_steps=1000,
  #save_steps=500,
  #eval_steps=500,
  logging_steps=500,
  #logging_steps=10,
  learning_rate=3e-4,
  #learning_rate=1e-4,
  warmup_steps=1000,
  save_total_limit=2
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=processor.feature_extractor,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import gc
collected = gc.collect()

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=60, training_loss=10.468480428059896, metrics={'train_runtime': 1274.2349, 'train_samples_per_second': 28.959, 'train_steps_per_second': 0.047, 'total_flos': 4.2154996665361295e+18, 'train_loss': 10.468480428059896, 'epoch': 24.0})

In [ ]:
trainer.push_to_hub()

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1716187270.f2330c546e04.19589.0:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

events.out.tfevents.1716186129.f2330c546e04.14734.0:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chaos6749/wav2vec2-large-xlsr-test/commit/0fc80c30b1c41d8b46f0a1c2eadf6cec451daf64', commit_message='End of training', commit_description='', oid='0fc80c30b1c41d8b46f0a1c2eadf6cec451daf64', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/wav2vec2-large-xlsr-test/wav2vec2-large-xlsr-test-trained_model")

In [ ]:
#processor = Wav2Vec2Processor.from_pretrained("chaos6749/wav2vec2-large-xlsr-test")
model = Wav2Vec2ForCTC.from_pretrained("chaos6749/wav2vec2-large-xlsr-test")

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

results = test_data.map(map_to_result, remove_columns=test_data.column_names)

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 1.000


In [ ]:
show_random_elements(results)

,pred_str,text
0,ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓ[UNK]ㅓㅈㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓ[UNK]ㅓㅓㅓㅓㅓㅓㅓㅓ[UNK]ㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅟㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅁㅓ,ㅅㅓㅇㄱㅘㅅㅏㅇㅇㅕㄱㅡㅁ ㅂㅜㄷㅏㅇㅅㅜㄹㅕㅇㅈㅏㅇㅢ ㅅㅜㄹㅕㅇㅇㅐㄱㅇㅡㄹ ㅎㅘㄴㅅㅜ ㅈㅗㅊㅣㅎㅏㄱㅗ ㄷㅏㅇㅡㅁㅎㅐ ㅅㅓㅇㄱㅘㅅㅏㅇㅇㅕㄱㅡㅁㅇㅡㄹ ㅈㅣㄱㅡㅂㅎㅏㅈㅣ ㅇㅏㄶㄴㅡㄴ ㅎㅏㄴㅍㅕㄴ ㅈㅣㅇㄱㅖ ㅈㅗㅊㅣㅎㅏㄱㅣㄹㅗ ㅎㅐㅆㄷㅏ
1,ㅓㅓㅓㄴㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓ[UNK]ㅓㅓㅓㅝㅓㅓㅓㅓㅓㅓㅓㅓㅡㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅡㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㄴㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ,ㅇㅏㄴ ㅇㅢㅇㅝㄴㅇㅢ ㅅㅓㄴㅈㅣㄴㅎㅘㅂㅓㅂ ㄱㅐㅈㅓㅇ ㅊㅏㄴㅅㅓㅇ ㅈㅜㅈㅏㅇㅇㅡㄹㅗ ㅎㅕㄴㅈㅐ ㅈㅓㅇㄱㅜㄱ ㅇㅣㅅㅠㄹㅗ ㅂㅜㅅㅏㅇㅎㅏㄴ ㅅㅓㄴㅈㅣㄴㅎㅘㅂㅓㅂ ㄴㅗㄴㅇㅢㅇㅔ ㅂㅕㄴㅎㅘㄱㅏ ㄷㅟㄸㅏㄹㅡㄹㅈㅣ ㅈㅜㅁㅗㄱㄷㅚㄴㄷㅏ
2,ㅓㅓㅓㄵㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㄴㅓㅓㅓㅓㅓㄵㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㄴㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ,ㅇㅚㄱㅜㄱㅇㅣㄴ ㅈㅣㅎㅟㅈㅏ ㅇㅕㅇㅇㅣㅂㄷㅗ ㅂㅏㅇㅇㅏㄴㅇㅣㄱㅣㄴㅡㄴ ㅎㅏㄴㅏ ㅇㅣ ㅂㅜㅂㅜㄴㅇㅡㄴ ㄴㅗㄴㅇㅢㅇㅔㅅㅓ ㅅㅓㄴㅇㅣㅁㄲㅏㅈㅣ ㅅㅏㅇㄷㅏㅇㅎㅏㄴ ㅅㅣㅇㅣㄹㅇㅣ ㄱㅓㄹㄹㅣㄴㅡㄴ ㅅㅓㄴㅌㅐㄱㅈㅣㄷㅏ
3,ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅡㅓㅓㅓㅓㅓㅓㅓ[UNK]ㅓㅓㅟㅓㅌㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅌㅟㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓ[UNK]ㅓㅓㅓㅌㅓㅓㅓㅓㅓㅍㅌㅓㅌㅓㅓㅓㅌㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓ[UNK]ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓ,ㄱㅏㅂㅣ ㅎㅗㄹㅈㅡㅇㅝㅅㅡㄴㅡㄴ ㅎㅕㄴㅈㅣ ㅁㅐㅊㅔ ㄷㅣ ㅇㅣㄴㅍㅗㅁㅔㅇㅣㅅㅕㄴㄱㅘㅇㅢ ㅇㅣㄴㅌㅓㅂㅠㅇㅔㅅㅓ ㅋㅐㄹㄹㅓㄴㅣㄱㅇㅣ ㅎㅚㅅㅏ ㄱㅗㅇㅟ ㅇㅣㅁㅇㅝㄴ ㄷㅏㅅㅓㅅ ㅁㅕㅇㄱㅘ ㅇㅣㅊㅓㄴ ㅅㅣㅂ ㅅㅏ ㄴㅕㄴ ㅈㅜㅇㅂㅏㄴ ㅊㅜㄹㅈㅏㅇ ㅊㅏ ㅅㅓㅇㅜㄹㅇㅡㄹ ㅊㅏㅈㅇㅏㅆㅇㅡㄹ ㄸㅐ ㅇㅔㅅㅡㅋㅗㅌㅡ ㅂㅏㅇㅔ ㄱㅏㅆㄷㅏㄱㅗ ㅁㅏㄹㅎㅐㅆㄷㅏ
4,,ㅅㅣㅂ ㅇㅣ ㅇㅣㄹ ㄱㅕㅇㅈㅜㅇㅔㅅㅓ ㅂㅏㄹㅅㅐㅇㅎㅏㄴ ㅈㅣㅈㅣㄴㅇㅡㄹㅗ ㅍㅗㅎㅏㅇㅅㅣ ㄴㅏㅁㄱㅜㅇㅢ ㅎㅏㄴ ㅁㅏㅌㅡ ㅈㅣㄴㅇㅕㄹㅈㅏㅇㅇㅔ ㅆㅏㅎㅇㅕ ㅇㅣㅆㄷㅓㄴ ㅅㅏㅇㅍㅜㅁㄷㅡㄹㅇㅣ ㅂㅏㄷㅏㄱㅇㅔ ㄸㅓㄹㅇㅓㅈㅕ ㅇㅣㅆㄷㅏ
5,ㅓㅓㅓㅓㅓㅓㅁㅓㅓㅓㅓㅓㅓㅓㅙㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ<s>ㅓㅓㅓㅓ[UNK]ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅘㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅘㅌㅓㅓㅓㅓㅓㅓㅓㅓㅟㅓㅓㅓㅌㅓㅓㅓㅓㅓㅈㅓㅓㅓㅓㅓㅓㅑㅓㅓㅓㅓㅓㅓ,ㅂㅏㄱㅈㅜㄴㅇㅕㅇ ㄷㅗㄴ ㅇㅏㄴ ㄷㅚㄴㅡㄴ ㄱㅗㅇㅇㅣㄱㅅㅏㄱㅓㄴ ㄴㅏㅁㄷㅡㄹ ㄷㅏ ㅂㅏㄱㅅㅜㅊㅕㅈㅜㄴㅡㄴ ㅅㅏㄱㅓㄴㅇㅡㄴ ㅈㅔㄱㅏ ㅎㅏㄱㅗ ㄷㅏㅇㅅㅣㄴㄴㅔㄷㅡㄹㅇㅡㄴ ㅅㅏㅁㅜㅅㅣㄹ ㅇㅜㄴㅇㅕㅇㅎㅐㅇㅑ ㄷㅚㄴㅣㄲㅏ ㄷㅗㄴ ㅂㅓㄴㅡㄴ ㅅㅏㄱㅓㄴ ㄷㅏㅇㅅㅣㄴㄴㅔㄷㅡㄹㅇㅣ ㅎㅐㄹㅏ
6,ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ,ㄷㅗㅅㅣㅇㅢ ㅅㅏㄹㅏㅁㄷㅡㄹㅇㅡㄴ ㄴㅏㅁㅜㅇㅘ ㅈㅏㅇㅝㄴㅇㅡㄹ ㅎㅘㄱㅂㅗㅎㅏㄱㅣ ㅇㅟㅎㅐ ㅇㅏㅍㅡㄹㅣㅋㅏ ㅁㅣㄹㄹㅣㅁㅇㅡㄹ ㅍㅏㄱㅚㅎㅐ ㄴㅏㄱㅏㅆㄷㅏ
7,ㅓㄵㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅈㅓㅓㅁㅓㅓㅌㅓㅓㅓ[UNK]ㅓㅓㅈㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅌㅓㅓㅈㅓㅓㅓㅓㅓㅓㅓㅓㅓㅡㅓㅓㅓㅓㅓㅓㅄㅓㅓㅓㅓㄵㅓㅌㅓㅁㅓㅈㅓㅓㅌㅓㅈㅓㅓㅟㅗㅓㅈㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㄴㅓㅓㅓㅓㅌㅓㅈㅓㅓㅈ,ㅎㅕㄴㅈㅐ ㅈㅜㄹㅅㅏㅌㅐㄱ ㅇㅕㄷㅡㄴ ㅇㅣㄹㄱㅗㅂ ㅊㅐ ㅈㅜㅇ ㅊㅣㄹㅅㅣㅂ ㅇㅕ ㅊㅐㄱㅏ ㅂㅣㄴ ㅅㅏㅇㅌㅐㅇㅕㅅㅓ ㅅㅏㅌㅐㄱ ㄷㅐㅂㅜㅂㅜㄴㅇㅣ ㅊㅓㄹㄱㅓㄷㅚㄹ ㄱㅏㄴㅡㅇㅅㅓㅇㅇㅣ ㅋㅡㄷㅏ
8,ㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅟㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅡㅓㅓㅓㅓㅓㅈㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅈㅓㅓㅓㅌㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㄴㅓㅓ[UNK]ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅌㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅌㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅁㅓㄵㅓㅓㅓㅓㅓㅓㅓㅈㅓㅓㅈㅓㅌㅌㅓㅓㅓㅓㅘㅓㅓㅓㅓㅓㅓㅓㅓㅈㅓㅈㅓ,ㅊㅓㅅㅂㅓㄴㅉㅐㄴㅡㄴ ㅇㅔㄹㄴㅣㄴㅛㄹㅗ ㅇㅣㄴㅎㅐ ㅎㅐㅅㅜㅁㅕㄴ ㅇㅗㄴㄷㅗㄱㅏ ㄴㅗㅍㅇㅏ ㄷㅐㄹㅠ ㅎㅕㄴㅅㅏㅇㅇㅣ ㄱㅏㅇㅎㅏㄱㅔ ㅂㅏㄹㄷㅏㄹㅎㅏㄴㅡㄴ ㄱㅓㅅㅇㅣㄱㅗ ㄷㅜㅂㅓㄴㅉㅐㄴㅡㄴ ㅅㅏㅇㅊㅡㅇ ㄱㅗㄱㅣㅇㅏㅂㅇㅣ ㅂㅏㄹㄷㅏㄹㅎㅏㅁㅕㄴㅅㅓ ㄷㅐㄹㅠ ㅎㅕㄴㅅㅏㅇㅇㅡㄹ ㄷㅓ ㄱㅏㅇㅎㅘㅅㅣㅋㅣㄴㅡㄴ ㅇㅕㄱㅎㅏㄹㅇㅡㄹ ㅎㅏㄱㅗ ㅇㅣㅆㅅㅡㅂㄴㅣㄷㅏ
9,ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅌㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ,ㅅㅣㅂ ㅇㅣㄹ ㅇㅣㅅㅣㅂ ㅅㅏ ㅇㅣㄹ ㅎㅗㅇㅋㅗㅇㅎㅐㅇㅇㅡㄴ ㅂㅣㅎㅐㅇㅅㅣㄱㅏㄴㅇㅣ ㅇㅘㅇㅂㅗㄱ ㅇㅣㄹㄱㅗㅂ ㅅㅣㄱㅏㄴㅇㅡㄹㅗ ㅇㅕㄴㅡ ㄷㅏㄴㄱㅓㄹㅣㅂㅗㄷㅏ ㄱㅣㄹㅇㅓ ㅅㅡㅇㅁㅜㅇㅝㄴ ㅅㅏㅇㅣㅇㅔㅅㅓ ㅎㅣㅁㄷㅡㄹㄷㅏㄱㅗ ㅅㅗㅁㅜㄴㄴㅏ ㅇㅣㅆㄷㅏ
